In [11]:
# import needed libraries
import pandas as pd
import numpy as np
import re
import gzip
import datetime
%matplotlib inline

import matplotlib.pyplot as plt
import scipy.stats as stats

data_path = './Data/MovieSummaries/'

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

C:\Users\Jan\AppData\Local\Temp/ipykernel_17460/2793029989.py:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [12]:
# import the movie metadata
filename = 'movie.metadata.tsv'
df_mov = pd.read_csv(data_path + filename, sep='\t', header=None)
df_mov = df_mov.rename(columns={0: "Wikipedia_movie_ID", 1: "Freebase_movie_ID", 2: "Movie_name", 3: "Movie_release_date", 4: "Movie_box_office_revenue", 5: "Movie_runtime", 6: "Movie_languages_(Freebase_ID:name_tuples)", 7: "Movie_countries_(Freebase_ID:name_tuples)", 8: "Movie_genres_(Freebase_ID:name_tuples)"})
print((df_mov.shape))
df_mov.head(20)

(81741, 9)


,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages_(Freebase_ID:name_tuples),Movie_countries_(Freebase_ID:name_tuples),Movie_genres_(Freebase_ID:name_tuples)
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science Fiction"", ""/m/03npn"": ""Horror"", ""/m/03k9fj"": ""Adventure"", ""/m/0fdjb"": ""Supernatural"", ""/m/02kdv5l"": ""Action"", ""/m/09zvmj"": ""Space western""}"
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey Mystery,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0"": ""Drama"", ""/m/0hj3n01"": ""Crime Drama""}"
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""Drama""}"
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic thriller"", ""/m/09blyk"": ""Psychological thriller""}"
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
5,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indie"", ""/m/01g6gs"": ""Black-and-white"", ""/m/01z4y"": ""Comedy""}"
6,18998739,/m/04jcqvw,The Sorcerer's Apprentice,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0hzlz"": ""South Africa""}","{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fantasy"", ""/m/03k9fj"": ""Adventure"", ""/m/03q4nz"": ""World cinema""}"
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04t36"": ""Musical"", ""/m/01z4y"": ""Comedy"", ""/m/01g6gs"": ""Black-and-white""}"
8,9997961,/m/06_y2j7,Contigo y aquí,1974,NaN,NaN,"{""/m/06nm1"": ""Spanish Language""}","{""/m/0jgd"": ""Argentina""}","{""/m/04t36"": ""Musical"", ""/m/07s9rl0"": ""Drama"", ""/m/01z4y"": ""Comedy""}"
9,2345652,/m/075f66,City of the Dead,1960,NaN,76.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/03npn"": ""Horror"", ""/m/0fdjb"": ""Supernatural""}"


In [23]:
# import the character metadata
filename = 'character.metadata.tsv'
df_char = pd.read_csv(data_path + filename, sep='\t', header=None)
df_char = df_char.rename(columns={0: "Wikipedia_movie_ID", 1: "Freebase_movie_ID", 2: "Movie_release_date", 3: "Character_name", 4: "Actor_date_of_birth", 5: "Actor_gender", 6: "Actor_height_(in_meters)", 7: "Actor_ethnicity_(Freebase_ID)", 8: "Actor_name", 9: "Actor_age_at_movie_release", 10: "Freebase_character/actor_map_ID", 11: "Freebase_character_ID", 12: "Freebase_actor_ID"})
print((df_char.shape))
df_char.head(20)
# df_char['Actor_ethnicity_(Freebase_ID)'].unique()

(450669, 13)


array([nan, '/m/044038p', '/m/0x67', '/m/064b9n', '/m/041rx', '/m/033tf_',
       '/m/04gfy7', '/m/0222qb', '/m/01qhm_', '/m/0dryh9k', '/m/048sp5',
       '/m/04mvp8', '/m/0bzkm2', '/m/02p1pl6', '/m/0bjbszh', '/m/022fdt',
       '/m/0cqgdq', '/m/0ffkb4', '/m/075dhf0', '/m/01hwt', '/m/0xnvg',
       '/m/0dqqwy', '/m/048z7l', '/m/07bch9', '/m/09v5bdn', '/m/02w7gg',
       '/m/03bkbh', '/m/02vsw1', '/m/09kr66', '/m/09vc4s', '/m/0g0x7_',
       '/m/042gtr', '/m/0cm7w1', '/m/046cwm', '/m/04dbw3', '/m/02ctzb',
       '/m/0g8_vp', '/m/092h2qt', '/m/0g6ff', '/m/0278pqj', '/m/0301y_',
       '/m/019kn7', '/m/0cnvdq1', '/m/03295l', '/m/065b6q', '/m/03pqwy',
       '/m/01xhh5', '/m/03ts0c', '/m/06gbnc', '/m/07hwkr', '/m/0bpjh3',
       '/m/0fpjs3j', '/m/04nrnz', '/m/09k5jvk', '/m/07mqps', '/m/08hpk0',
       '/m/03ttfc', '/m/0d9q7j', '/m/075_n6', '/m/0dllcfn', '/m/04kbvpz',
       '/m/03ftx7', '/m/0747611', '/m/025rpb0', '/m/06mvq', '/m/047l_90',
       '/m/029f2r', '/m/01rv7x', '/m/05sf2x', '/m/

In [14]:
# import the plot summary into a list
filename = 'plot_summaries.txt'
with open(data_path + filename, encoding='ISO-8859-1') as file:
    plot = [line.rstrip() for line in file]

# print the first 5 plot summaries
print(len(plot))
plot[0:3]

42306


["23890098\tShlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.",
 '31186339\tThe nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker\'s son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively at special academie

In [15]:
# clean the movie dataframe -> delete all movies, which have no information about their release date or their box office revenue
df_mov_orig = df_mov
print("Original size df_mov:", df_mov_orig.shape)
df_mov = df_mov.dropna(subset=['Movie_release_date', 'Movie_box_office_revenue'])
print("Size of cleaned df_mov:", df_mov.shape)
print(len(df_mov_orig)-len(df_mov), "movies discarded, because they had no information about their release date or their box office revenue")

Original size df_mov: (81741, 9)
Size of cleaned df_mov: (8328, 9)
73413 movies discarded, because they had no information about their release date or their box office revenue


In [16]:
# Convert all date strings to datetime objects
# TODO: correct all the dates that were represented just as a year (e.g. 1988) and are now represented as the first day of the year (e.g. 1988-01-01)
df_mov['Movie_release_date'] = pd.to_datetime(df_mov['Movie_release_date'], errors = 'coerce')
# clean movies where the Movie release date could not be transformed from a string into a datetime object
df_mov_old = df_mov
df_mov = df_mov.dropna(subset=['Movie_release_date'])
print("Size of cleaned df_mov:", df_mov.shape)
print(len(df_mov_old)-len(df_mov), "movies discarded, because the Movie release date could not be transformed from a string into a datetime object")

Size of cleaned df_mov: (8327, 9)
1 movies discarded, because the Movie release date could not be transformed from a string into a datetime object


C:\Users\Jan\AppData\Local\Temp/ipykernel_17460/2587505201.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mov['Movie_release_date'] = pd.to_datetime(df_mov['Movie_release_date'], errors = 'coerce')


In [17]:
# sort movie dataframe after the movie release date
df_mov = df_mov.sort_values(by=['Movie_release_date'], ascending=True)
print(df_mov.head())

       Wikipedia_movie_ID Freebase_movie_ID                     Movie_name  \
48417  28703057            /m/0czdh_n        The Corbett-Fitzsimmons Fight   
50991  1762869             /m/05tz4w         The Squaw Man                   
53319  15340505            /m/03m5kln        Neptune's Daughter              
49852  15130915            /m/03hjhh9        The Million Dollar Mystery      
1026   14613957            /m/03gqmqv        Rose of the Rancho              

      Movie_release_date  Movie_box_office_revenue  Movie_runtime  \
48417 1897-05-22          100000.0                 NaN              
50991 1914-02-15          244700.0                  74.0            
53319 1914-04-25          1000000.0                NaN              
49852 1914-06-22          1500000.0                NaN              
1026  1914-11-15          87028.0                  NaN              

                           Movie_languages_(Freebase_ID:name_tuples)  \
48417  {}                                   

In [18]:
# build a dataframe which contains the Wikipedia movie ID of the most sucessful movie of every year
# dft1 = df1.query('date_created >= "2013-04-01" and date_created <= "2014-03-31"')
year = 1897
list_max_suc_year = []
while year < 2013:
    # first day of given year
    ref_date_start = str(datetime.datetime(year, 1, 1))
    # last day of given year
    ref_date_end = str(datetime.datetime(year, 12, 31))
    # select all movies of given year
    df_year = df_mov.query('Movie_release_date >= @ref_date_start and Movie_release_date <= @ref_date_end')
    df_year = df_year.sort_values(by=['Movie_box_office_revenue'], ascending=False)
    if df_year.empty == False:
        # add Wikipedia movie ID to the list
        list_max_suc_year.append(df_year.iloc[0,0])
    year = year + 1

print(len(list_max_suc_year))
print(list_max_suc_year)

96
[28703057, 15130915, 3333, 4499436, 2137591, 29299071, 3202250, 3371111, 1343327, 25442375, 68145, 1995966, 9251956, 1282851, 73488, 29034714, 3947300, 44696, 3992212, 74962, 76361, 62084, 2804704, 187010, 4837038, 683122, 74862, 73438, 79729, 61141, 17995856, 76297, 447194, 399402, 154790, 165369, 982199, 74577, 286828, 76383, 42856, 11073939, 1343342, 166813, 179351, 1830725, 210224, 6322029, 4836239, 9377241, 680006, 23941708, 99031, 11577392, 157481, 2466773, 725459, 73875, 74830, 45772, 52549, 336846, 6446062, 53964, 54166, 73441, 50744, 81503, 42993, 179825, 129295, 129368, 51888, 129619, 34344124, 73170, 68485, 88678, 357473, 52389, 52371, 52390, 50793, 451866, 667361, 173944, 174251, 525314, 667372, 1908238, 1689394, 4276475, 4273140, 1213838, 31941988, 22114132]


In [20]:
# use the list of the Wikipedia movie IDs of the most sucessful movies of every year to create a dataframe of these movies
df_suc = pd.DataFrame()
for idx in range(0,len(list_max_suc_year)):
    movieID = list_max_suc_year[idx]
    df_suc_year = df_mov.query('Wikipedia_movie_ID == @movieID')
    df_suc = df_suc.append(df_suc_year, ignore_index=True)
df_suc
    

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages_(Freebase_ID:name_tuples),Movie_countries_(Freebase_ID:name_tuples),Movie_genres_(Freebase_ID:name_tuples)
0,28703057,/m/0czdh_n,The Corbett-Fitzsimmons Fight,1897-05-22,1.000000e+05,NaN,{},{},"{""/m/01z02hx"": ""Sports""}"
1,15130915,/m/03hjhh9,The Million Dollar Mystery,1914-06-22,1.500000e+06,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indie"", ""/m/01g6gs"": ""Black-and-white"", ""/m/03k9fj"": ""Adventure""}"
2,3333,/m/0151l,The Birth of a Nation,1915-01-01,5.000000e+07,190.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indie"", ""/m/04xvh5"": ""Costume drama"", ""/m/06l3bl"": ""Epic"", ""/m/01g6gs"": ""Black-and-white"", ""/m/07s9rl0"": ""Drama"", ""/m/082gq"": ""War film""}"
3,4499436,/m/0c5ppr,Way Down East,1920-01-01,4.500000e+06,147.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indie"", ""/m/01g6gs"": ""Black-and-white"", ""/m/03btsm8"": ""Action/Adventure"", ""/m/0hfjk"": ""Western"", ""/m/07s9rl0"": ""Drama"", ""/m/068d7h"": ""Romantic drama"", ""/m/02l7c8"": ""Romance Film""}"
4,2137591,/m/06pkq3,The Four Horsemen of the Apocalypse,1921-03-06,4.500000e+06,134.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indie"", ""/m/01g6gs"": ""Black-and-white"", ""/m/07s9rl0"": ""Drama"", ""/m/082gq"": ""War film"", ""/m/02l7c8"": ""Romance Film""}"
5,29299071,/m/0ds41g2,In the Name of the Law,1922-07-09,1.753097e+07,NaN,{},"{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}"
6,3202250,/m/08yzpw,The Ten Commandments,1923-01-01,4.168790e+06,146.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04p5fxn"": ""Christian film"", ""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indie"", ""/m/04xvh5"": ""Costume drama"", ""/m/06l3bl"": ""Epic"", ""/m/01g6gs"": ""Black-and-white"", ""/m/07s9rl0"": ""Drama""}"
7,3371111,/m/0983w1,Girl Shy,1924-04-19,1.550000e+06,80.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indie"", ""/m/0hj3mxr"": ""Chase Movie"", ""/m/01g6gs"": ""Black-and-white"", ""/m/01z4y"": ""Comedy"", ""/m/02l7c8"": ""Romance Film""}"
8,1343327,/m/04v8w5,Ben-Hur,1925-12-30,1.073800e+07,143.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/07s9rl0"": ""Drama"", ""/m/0219x_"": ""Indie"", ""/m/06l3bl"": ""Epic""}"
9,25442375,/m/09k7ddx,Aloma of the South Seas,1926-05-16,3.000000e+06,90.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film""}"
